In [18]:
import numpy as np
import json
from pprint import pprint
from datetime import datetime
from scipy.interpolate import interp1d

In [19]:
# The hourly temperature from 9:00 to 12:00 on the day when the data for this study was collected was used for correction.
# The temperature was provided by the Meteorological Bureau. 
# Follow-up studies could conduct localized temperature measurements to improve calibration accuracy.

weather_time = np.array([1691802000.0, 1691805600.0, 1691809200.0, 1691812800.0, 1691816400.0])
weather_temp = np.array([30.0, 31.0, 32.9, 34.5, 31.1])

In [20]:
with open('data\data.json') as json_file:
    c_data = json.load(json_file)

In [21]:
my_time = []
my_temp = []

In [22]:
for cd in c_data:
    format_str = "%Y-%m-%d %H:%M:%S"
    datetime_obj = datetime.strptime(cd['date2'], format_str)
    timestamp = datetime_obj.timestamp()
    my_time.append(timestamp)
    my_temp.append(float(cd['temp']))

In [23]:
data_time = np.array(my_time)
data_temp = np.array(my_temp)

In [24]:
len(my_time)

4655

In [25]:
#Correction using one-dimensional interpolation

interpolator = interp1d(weather_time, weather_temp, kind='linear', fill_value="extrapolate")
interpolated_temperature = interpolator(my_time)

In [26]:
calied = my_temp / (interpolated_temperature / interpolated_temperature[0])
calied

array([36.318     , 36.336     , 36.3173827 , ..., 33.68768341,
       33.73597365, 33.73597365])

In [27]:
calied_data = []

In [28]:
for i in range(len(c_data)):
    cd = c_data[i]
    cd['cali_temp'] = calied[i]
    calied_data.append(cd)

In [29]:
calied_data[0]

{'date': '2023-08-12 10:01:36.223',
 'date2': '2023-08-12 10:01:36',
 'temp': '36.318',
 'rh': '75.253',
 'atm': '100.75590515',
 'lat': '25.038885',
 'lon': '121.530971',
 'rel_alt': '-0.000',
 'cali_temp': 36.318}

In [30]:
len(calied_data)

4655

In [32]:
with open('data/calied_data.json', 'w') as json_file:
    json.dump(calied_data, json_file, indent=4)